In [23]:
import os
from dotenv import load_dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
from pinecone import Pinecone, ServerlessSpec


In [13]:
load_dotenv()

# Step 1: Load and preprocess PDF data
def load_pdf(data_path):
    loader = DirectoryLoader(data_path, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

extracted_data = load_pdf("data/")

def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

text_chunks = text_split(extracted_data)
print("Number of chunks:", len(text_chunks))

Number of chunks: 5860


In [15]:
# Step 2: Initialize embeddings
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

embeddings = download_hugging_face_embeddings()

c:\Gen_AI_Projects\chatbot\my_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Gen_AI_Projects\chatbot\my_venv\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\krishna_gskr\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In 

In [25]:
# Step 3: Configure Pinecone
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
INDEX_NAME = os.getenv("INDEX_NAME")
# Number of dimensions in your embedding model
DIMENSION = int(os.getenv("DIMENSION"))  

# Initialize Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)

# Check if the index exists and create it only if it doesn't
try:
    existing_indexes = [index.name for index in pc.list_indexes()]
    if INDEX_NAME in existing_indexes:
        print(f"Index '{INDEX_NAME}' already exists.")
    else:
        print(f"Index '{INDEX_NAME}' does not exist. Creating index...")
        pc.create_index(
            name=INDEX_NAME,
            dimension=DIMENSION,
            metric="cosine",
            spec=ServerlessSpec(
                cloud="aws",
                region="us-east-1"
            )
        )
        print(f"Index '{INDEX_NAME}' created successfully.")
except Exception as e:
    print(f"Error during index management: {e}")

# Connect to the existing or newly created index
try:
    index = pc.index(INDEX_NAME)
    print(f"Connected to index '{INDEX_NAME}'.")
except Exception as e:
    print(f"Error connecting to index '{INDEX_NAME}': {e}")

Index 'chatbot' does not exist. Creating index...
Index 'chatbot' created successfully.
Error connecting to index 'chatbot': 'Pinecone' object has no attribute 'index'
